In [2]:
import pandas as pd
import pyodbc
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil.rrule import rrule, MONTHLY
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import plotly.express as px
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import shap
import numdifftools as nd
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope
shap.initjs()
%load_ext autoreload
%autoreload 2
import mmm_transformations
import mmm_preprocessing
import mmm_modeling
import mmm_response_curves
import mmm_optimization

# Summary

In [223]:
# description of what this notebook does
# should be minimal code and jsut outputs

# Data Pull

In [2]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PRDSPPI10DB1;'
                      'Port=1433;')

In [3]:
sql_stmt = """SELECT	r.HCOId,
		hcoAddr.BaseZip AS HCOZip,
		r.MonthDesc,
		r.YearDesc,
		r.BrandName,
		r.Emails,
		r.Phones,
		r.FTF,
		r.Virtual,
		r.Mssg,
		r.Other,
		r.Quantity,
		fncl.*,
		hco_bed_total.*,
		hco_bus.*

FROM (
SELECT	ISNULL(a.HCOId, b.HCOId) AS HCOId,
		ISNULL(a.YearDesc, b.YearDesc) AS YearDesc,
		ISNULL(a.MonthDesc, b.MonthDesc) AS MonthDesc,
		ISNULL(a.BrandName, b.BrandName) AS BrandName,
		ISNULL(b.Emails, 0) AS Emails,
		ISNULL(b.Phones, 0) AS Phones,
		ISNULL(b.FTF, 0) AS FTF,
		ISNULL(b.Virtual, 0) AS Virtual,
		ISNULL(b.Mssg, 0) AS Mssg,
		ISNULL(b.Other, 0) AS Other,
		ISNULL(a.Quantity, 0) AS Quantity 
FROM (
	SELECT	s.HCOId, 
		d.YearDesc, 
		d.MonthDesc,
		p.BrandName,
		SUM(s.QuantityAdjusted) AS Quantity		
		FROM meas.tblFactSalesTransaction s
		INNER JOIN meas.tblDimDate d ON s.SalesTransactionDateId = d.DateId
		INNER JOIN	meas.tblDimProduct p	ON	s.ProductId = p.ProductId
		WHERE s.HCOId <> -1
		GROUP BY s.HCOId, 
				d.YearDesc, 
				d.MonthDesc,
				p.BrandName
) a
/* ProductId =1 (Rolvedon). All ProductId in FasctSalesTrasaction equals to 1 
 FactSalesTransaction (hco-date level) has 2819 rows and
 a (hco-month level) has 945 rows*/ 


FULL OUTER JOIN (
SELECT
    fc.HCOId, 
    d.YearDesc,
    d.MonthDesc,
    p.BrandName,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Email_vod' THEN fc.CallId END) AS Emails,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Phone_vod' THEN fc.CallId END) AS Phones,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Face_to_face_vod' THEN fc.CallId END) AS FTF,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Video_vod' THEN fc.CallId END) AS Virtual,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Message_vod' THEN fc.CallId END) AS Mssg,
    COUNT(DISTINCT CASE WHEN cc.CallChannelName = 'Other_vod' THEN fc.CallId END) AS Other
FROM
    (
        SELECT * 
		/*
            CASE
                WHEN HCPId <> -1 THEN HCPId
                ELSE HCOId
            END AS ChildId,
			CASE 
				WHEN HCPId <> -1 THEN 1
				ELSE 2
			END AS AffiliationTypeId */
        FROM meas.tblFactCall
		WHERE HCOId <> -1
/* It is confirmed that FactCAll is either HCPId or HCOId level 
HCOId = -1 has 8173 rows; HCO<>-1 has 5685. tblFactCall has a total of 13858 rows*/         
    ) fc
/*LEFT JOIN
    meas.tblDimAffiliation da ON fc.ChildId = da.ChildId AND fc.AffiliationTypeId=da.AffiliationTypeId and da.[Primary]=1 and da.Active=1
	 type=1, HCP-HCO, type=2, HCO-HCO  */

INNER JOIN
    dbo.tblDimCallChannel cc ON fc.CallChannelId = cc.CallChannelId
INNER JOIN
    meas.tblDimDate d ON fc.CallDateId = d.DateId
INNER JOIN
    meas.tblFactCallDetail fcd ON fc.CallSourceId = fcd.CallSourceId
LEFT JOIN
    meas.tblDimProduct p ON fcd.ProductId = p.ProductId
GROUP BY
    fc.HCOId,
    d.YearDesc,
    d.MonthDesc,
    p.BrandName
) b 
/*FactCall has 13858 (date-level) rows and b (month level) has 2885 rows*/
ON a.HCOId = b.HCOId AND a.YearDesc = b.YearDesc
	AND a.MonthDesc = b.MonthDesc
	AND a.BrandName = b.BrandName
) r
INNER JOIN MEAS.tblDimHCO hco ON r.HCOId = hco.HCOId
LEFT OUTER JOIN 
(SELECT TPSEntityId as HCOSourceId, iq_hco_fncl.*
FROM [SPECTRUM_ADHOC].[dbo].[tblDataFeed_IQVIA_OneKey_HCO_FNCL_FACT] iq_hco_fncl
--tblDataFeed_IQVIA_OneKey_HCO_FNCL_FACT has 4149 rows
INNER JOIN (
    SELECT *
    FROM [SPECTRUM_CM].[CM].[tblOutletAlternateId]
    WHERE TPSDataFeedId = 6
) oa ON iq_hco_fncl.HCO_HCE_ID = oa.DataProviderUniqueIdentifier COLLATE SQL_Latin1_General_CP1_CI_AS
-- "SQL_Latin1_General_CP1_CI_AS" indicates to read collumn via "class insensitive"
  ) fncl ON hco.HCOSourceID = fncl.HCOSourceId 
--fncl only has 4149 rows

LEFT OUTER JOIN 
(SELECT TPSEntityId as HCOSourceId, bed.*
FROM [SPECTRUM_ADHOC].[dbo].[tblDataFeed_IQVIA_OneKey_HCO_BED_FACT] bed 
INNER JOIN (
    SELECT *
    FROM [SPECTRUM_CM].[CM].[tblOutletAlternateId]
    WHERE TPSDataFeedId = 6
) oa ON bed.HCO_HCE_ID = oa.DataProviderUniqueIdentifier COLLATE SQL_Latin1_General_CP1_CI_AS
WHERE BED_DESC = 'Total' 
  ) hco_bed_total ON hco.HCOSourceID = hco_bed_total.HCOSourceId 
 --hco_bed_total has 4725 rows


LEFT OUTER JOIN 
(SELECT TPSEntityId as HCOSourceId, bus.* FROM (SELECT
    HCO_HCE_ID,
    MAX(CASE WHEN DETL_CD = 'avgoccrate' THEN DETL_NBR END) AS avgoccrate,
    MAX(CASE WHEN DETL_CD = 'routineday' THEN DETL_NBR END) AS routineday,
    MAX(CASE WHEN DETL_CD = 'H_pdays' THEN DETL_NBR END) AS H_pdays,
    MAX(CASE WHEN DETL_CD = 'avglos' THEN DETL_NBR END) AS avglos,
    MAX(CASE WHEN DETL_CD = 'mparmix' THEN DETL_NBR END) AS mparmix,
    MAX(CASE WHEN DETL_CD = 'mcaredrg' THEN DETL_NBR END) AS mcaredrg,
    MAX(CASE WHEN DETL_CD = 'ttldrg' THEN DETL_NBR END) AS ttldrg,
    MAX(CASE WHEN DETL_CD = 'MCR_alos' THEN DETL_NBR END) AS MCR_alos,
    MAX(CASE WHEN DETL_CD = 'hdrg' THEN DETL_NBR END) AS hdrg,
    MAX(CASE WHEN DETL_CD = 'MCR_pdays' THEN DETL_NBR END) AS MCR_pdays
FROM
    [SPECTRUM_ADHOC].[dbo].[tblDataFeed_IQVIA_OneKey_HCO_BUS_DETL_FACT]
GROUP BY
    HCO_HCE_ID
HAVING
    COALESCE(MAX(CASE WHEN DETL_CD = 'avgoccrate' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'routineday' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'H_pdays' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'avglos' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'mparmix' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'mcaredrg' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'ttldrg' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'MCR_alos' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'hdrg' THEN DETL_NBR END),
             MAX(CASE WHEN DETL_CD = 'MCR_pdays' THEN DETL_NBR END)) IS NOT NULL
) bus
INNER JOIN (
    SELECT *
    FROM [SPECTRUM_CM].[CM].[tblOutletAlternateId]
    WHERE TPSDataFeedId = 6
) oa ON bus.HCO_HCE_ID = oa.DataProviderUniqueIdentifier COLLATE SQL_Latin1_General_CP1_CI_AS
) hco_bus ON hco.HCOSourceID = hco_bus.HCOSourceId 
--hco_bus has 4145 rows 

LEFT OUTER JOIN MEAS.tblDimEntityAddress hcoEntAddr
	ON hco.HCOId = hcoEntAddr.EntityId 
	AND hcoEntAddr.EntityType = 'HCO'
	AND hcoEntAddr.Active = 1
	AND hcoEntAddr.IsPrimary = 1
LEFT OUTER JOIN MEAS.tblDimAddress hcoAddr
	ON hcoEntAddr.AddressId = hcoAddr.AddressId
	AND hcoAddr.Active = 1
ORDER BY r.YearDesc DESC, r.MonthDesc DESC

/* MEAS.tblDimHCO has 53747 distinct id 
using the latest b query, the output of this query is now 8182 rows. (0719)
using the latest b query (consider only hco), the output of this query is now 3845 rows
using the latest b query (consider only hco) + fncl, the output of this query is now 290 rows */ 
"""

In [4]:
df = pd.read_sql(sql_stmt, conn)
df

DatabaseError: Execution failed on sql 'SELECT r.HCOId,
                     hcoAddr.BaseZip AS HCOZip,
                     r.MonthDesc,
                     r.YearDesc,
                     r.ProductGroupName,
                     r.Emails,
                     r.Phones,
                     r.FTF,
                     r.Virtual,
                     r.Mssg,
                     r.Other,
                     r.Quantity

              FROM (
              SELECT  ISNULL(a.HCOId, b.HCOId) AS HCOId,
                      ISNULL(a.YearDesc, b.YearDesc) AS YearDesc,
                      ISNULL(a.MonthDesc, b.MonthDesc) AS MonthDesc,
                      ISNULL(a.ProductGroupName, b.ProductGroupName) AS ProductGroupName,
                      ISNULL(b.Emails, 0) AS Emails,
                      ISNULL(b.Phones, 0) AS Phones,
                      ISNULL(b.FTF, 0) AS FTF,
                      ISNULL(b.Virtual, 0) AS Virtual,
                      ISNULL(b.Mssg, 0) AS Mssg,
              

# Preprocessing

In [3]:
df = pd.read_csv('spectrum_balanced.csv')
df

,SystemID,Date,BaseZip,SystemName,Emails,Phones,FTF,Virtual,Mssg,Other,...,avgoccrate,routineday,H_pdays,avglos,mparmix,mcaredrg,ttldrg,MCR_alos,hdrg,MCR_pdays
0,4769,2022-10,29732.0,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
1,4769,2022-11,29732.0,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",0.0,0.0,1.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
2,4769,2022-12,29732.0,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",0.0,1.0,1.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
3,4769,2023-01,29732.0,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",2.0,2.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
4,4769,2023-02,29732.0,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",1.0,2.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8775,44280,2023-03,78229.0,"SOUTH TEXAS ONCOLOGY & HEMATOLOGY, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
8776,44280,2023-04,78229.0,"SOUTH TEXAS ONCOLOGY & HEMATOLOGY, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
8777,44280,2023-05,78229.0,"SOUTH TEXAS ONCOLOGY & HEMATOLOGY, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
8778,44280,2023-06,78229.0,"SOUTH TEXAS ONCOLOGY & HEMATOLOGY, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8


In [4]:
len(np.unique(df['SystemID']))

878

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
SystemID,8780.0,2.859333e+04,1.005122e+04,4746.0000,2.286500e+04,27063.50,32535.00,5.405300e+04
BaseZip,8510.0,5.460129e+04,2.849412e+04,1605.0000,3.250300e+04,54303.00,78550.00,9.970100e+04
Emails,8780.0,3.439636e-01,1.332655e+00,0.0000,0.000000e+00,0.00,0.00,5.800000e+01
Phones,8780.0,1.420273e-01,6.852133e-01,0.0000,0.000000e+00,0.00,0.00,2.000000e+01
FTF,8780.0,1.178815e-01,5.612088e-01,0.0000,0.000000e+00,0.00,0.00,1.400000e+01
Virtual,8780.0,2.562642e-02,1.882873e-01,0.0000,0.000000e+00,0.00,0.00,4.000000e+00
Mssg,8780.0,1.343964e-02,1.793580e-01,0.0000,0.000000e+00,0.00,0.00,6.000000e+00
Other,8780.0,7.084282e-02,3.282527e-01,0.0000,0.000000e+00,0.00,0.00,6.000000e+00
Quantity_adj,8780.0,1.439294e+00,2.406048e+01,0.0000,0.000000e+00,0.00,0.00,1.614000e+03
DDS_IN_PD_CNT,8780.0,2.468131e+03,3.013296e+03,183.0000,1.456000e+03,2468.13,2468.13,4.208000e+04


In [6]:
df_pp = df.copy()
df_pp.columns = df_pp.columns.str.lower()
df_pp

,systemid,date,basezip,systemname,emails,phones,ftf,virtual,mssg,other,...,avgoccrate,routineday,h_pdays,avglos,mparmix,mcaredrg,ttldrg,mcr_alos,hdrg,mcr_pdays
0,4769,2022-10,29732.0,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
1,4769,2022-11,29732.0,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",0.0,0.0,1.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
2,4769,2022-12,29732.0,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",0.0,1.0,1.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
3,4769,2023-01,29732.0,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",2.0,2.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
4,4769,2023-02,29732.0,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",1.0,2.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8775,44280,2023-03,78229.0,"SOUTH TEXAS ONCOLOGY & HEMATOLOGY, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
8776,44280,2023-04,78229.0,"SOUTH TEXAS ONCOLOGY & HEMATOLOGY, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
8777,44280,2023-05,78229.0,"SOUTH TEXAS ONCOLOGY & HEMATOLOGY, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8
8778,44280,2023-06,78229.0,"SOUTH TEXAS ONCOLOGY & HEMATOLOGY, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,2.66,232387.72,232387.72,37.75,2.76,14664.21,57194.79,39.79,57194.79,68911.8


# Transformations

In [67]:
df_pp.corr()

,systemid,basezip,emails,phones,ftf,virtual,mssg,other,quantity_adj,dds_in_pd_cnt,...,avgoccrate,routineday,h_pdays,avglos,mparmix,mcaredrg,ttldrg,mcr_alos,hdrg,mcr_pdays
systemid,1.000000,-0.043763,0.040585,-0.003854,-0.005933,0.018727,0.022427,0.000899,2.510066e-03,-0.009650,...,-0.012684,-0.004847,-0.004847,-0.011932,-0.010724,-0.005917,-0.006421,-0.012221,-0.006421,-0.005495
basezip,-0.043763,1.000000,0.064302,-0.035111,0.008994,-0.025476,-0.026787,-0.055290,-1.578954e-02,-0.019945,...,-0.034348,-0.057696,-0.057696,-0.016496,-0.021777,-0.059342,-0.046678,-0.018225,-0.046678,-0.071538
emails,0.040585,0.064302,1.000000,0.345419,0.388833,0.262664,0.045946,0.040375,1.211873e-01,0.053756,...,0.049353,0.037531,0.037531,0.030018,0.052461,0.042606,0.037680,0.040300,0.037680,0.041979
phones,-0.003854,-0.035111,0.345419,1.000000,0.315172,0.169554,0.070664,0.001347,7.865521e-02,0.052043,...,0.040839,0.026769,0.026769,0.033545,0.057516,0.037824,0.027914,0.044735,0.027914,0.036122
ftf,-0.005933,0.008994,0.388833,0.315172,1.000000,0.115857,0.069132,-0.005146,2.995243e-01,0.038774,...,0.034982,0.025000,0.025000,0.021496,0.038172,0.030603,0.025011,0.030556,0.025011,0.030338
virtual,0.018727,-0.025476,0.262664,0.169554,0.115857,1.000000,0.030276,0.031442,5.288126e-02,0.041169,...,0.036004,0.032677,0.032677,0.018461,0.038982,0.036516,0.030669,0.025047,0.030669,0.036033
mssg,0.022427,-0.026787,0.045946,0.070664,0.069132,0.030276,1.000000,-0.002630,7.289471e-03,-0.001199,...,-0.000701,-0.004596,-0.004596,-0.003560,-0.001024,-0.002960,-0.003520,-0.003611,-0.003520,-0.004110
other,0.000899,-0.055290,0.040375,0.001347,-0.005146,0.031442,-0.002630,1.000000,-1.151268e-02,0.148585,...,0.145650,0.133758,0.133758,0.076660,0.141513,0.143625,0.139864,0.100131,0.139864,0.136486
quantity_adj,0.002510,-0.015790,0.121187,0.078655,0.299524,0.052881,0.007289,-0.011513,1.000000e+00,-0.000085,...,-0.000060,0.000177,0.000177,-0.000112,-0.000103,0.000073,0.000122,-0.000106,0.000122,0.000149
dds_in_pd_cnt,-0.009650,-0.019945,0.053756,0.052043,0.038774,0.041169,-0.001199,0.148585,-8.508558e-05,1.000000,...,0.973564,0.862991,0.862991,0.715139,0.972840,0.885615,0.863585,0.856246,0.863585,0.882595


In [7]:
transform = mmm_transformations.MMMTransformations()

In [8]:
df_pp.describe()

,systemid,basezip,emails,phones,ftf,virtual,mssg,other,quantity_adj,dds_in_pd_cnt,...,avgoccrate,routineday,h_pdays,avglos,mparmix,mcaredrg,ttldrg,mcr_alos,hdrg,mcr_pdays
count,8780.000000,8510.000000,8780.000000,8780.000000,8780.000000,8780.000000,8780.000000,8780.000000,8780.000000,8780.000000,...,8780.000000,8.780000e+03,8.780000e+03,8780.000000,8780.000000,8780.000000,8.780000e+03,8780.000000,8.780000e+03,8.780000e+03
mean,28593.328018,54601.286722,0.343964,0.142027,0.117882,0.025626,0.013440,0.070843,1.439294,2468.131310,...,2.658833,2.323877e+05,2.323877e+05,37.749661,2.759348,14664.210626,5.719479e+04,39.788235,5.719479e+04,6.891180e+04
std,10051.218360,28494.122385,1.332655,0.685213,0.561209,0.188287,0.179358,0.328253,24.060477,3013.296154,...,3.235790,2.788143e+05,2.788143e+05,63.987459,3.430046,17629.717248,7.158559e+04,56.341959,7.158559e+04,8.001317e+04
min,4746.000000,1605.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,183.000000,...,0.036300,5.530000e+02,5.530000e+02,1.522800,0.003700,5.000000,7.600000e+01,1.000000,7.600000e+01,1.700000e+01
25%,22865.000000,32503.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1456.000000,...,1.487900,1.759000e+05,1.759000e+05,18.521900,1.422500,9998.000000,4.228800e+04,22.000000,4.228800e+04,4.807800e+04
50%,27063.500000,54303.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2468.130000,...,2.660000,2.323877e+05,2.323877e+05,37.750000,2.760000,14664.210000,5.719479e+04,39.790000,5.719479e+04,6.891180e+04
75%,32535.000000,78550.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2468.130000,...,2.660000,2.323877e+05,2.323877e+05,37.750000,2.760000,14664.210000,5.719479e+04,39.790000,5.719479e+04,6.891180e+04
max,54053.000000,99701.000000,58.000000,20.000000,14.000000,4.000000,6.000000,6.000000,1614.000000,42080.000000,...,50.049800,5.414923e+06,5.414923e+06,1238.712500,49.508900,334041.000000,1.424002e+06,1069.000000,1.424002e+06,1.466733e+06


In [132]:
df_t = transform.winsorize(df_pp, ['quantity_adj', 'emails', 'phones', 'ftf', 'virtual', 'mssg', 'other'], 99.9)
df_t.describe()

,systemid,basezip,emails,phones,ftf,virtual,mssg,other,quantity_adj,dds_in_pd_cnt,...,avgoccrate,routineday,h_pdays,avglos,mparmix,mcaredrg,ttldrg,mcr_alos,hdrg,mcr_pdays
count,8780.000000,8510.000000,8780.000000,8780.000000,8780.000000,8780.000000,8780.000000,8780.000000,8780.000000,8780.000000,...,8780.000000,8.780000e+03,8.780000e+03,8780.000000,8780.000000,8780.000000,8.780000e+03,8780.000000,8.780000e+03,8.780000e+03
mean,28593.328018,54601.286722,0.336105,0.138383,0.113894,0.024715,0.011276,0.069590,1.130978,2468.131310,...,2.658833,2.323877e+05,2.323877e+05,37.749661,2.759348,14664.210626,5.719479e+04,39.788235,5.719479e+04,6.891180e+04
std,10051.218360,28494.122385,1.148857,0.614113,0.495614,0.175254,0.130664,0.313438,11.395580,3013.296154,...,3.235790,2.788143e+05,2.788143e+05,63.987459,3.430046,17629.717248,7.158559e+04,56.341959,7.158559e+04,8.001317e+04
min,4746.000000,1605.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,183.000000,...,0.036300,5.530000e+02,5.530000e+02,1.522800,0.003700,5.000000,7.600000e+01,1.000000,7.600000e+01,1.700000e+01
25%,22865.000000,32503.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1456.000000,...,1.487900,1.759000e+05,1.759000e+05,18.521900,1.422500,9998.000000,4.228800e+04,22.000000,4.228800e+04,4.807800e+04
50%,27063.500000,54303.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2468.130000,...,2.660000,2.323877e+05,2.323877e+05,37.750000,2.760000,14664.210000,5.719479e+04,39.790000,5.719479e+04,6.891180e+04
75%,32535.000000,78550.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2468.130000,...,2.660000,2.323877e+05,2.323877e+05,37.750000,2.760000,14664.210000,5.719479e+04,39.790000,5.719479e+04,6.891180e+04
max,54053.000000,99701.000000,15.000000,9.000000,6.221000,2.000000,2.000000,3.000000,252.221000,42080.000000,...,50.049800,5.414923e+06,5.414923e+06,1238.712500,49.508900,334041.000000,1.424002e+06,1069.000000,1.424002e+06,1.466733e+06


In [115]:
#df_t = df_pp.copy()

In [110]:
df_t.to_csv('spectrum_balanced_winsorize.csv', index=False)

In [133]:
df_t = transform.lag_dv(df_t, 'quantity_adj', 3, 'systemid')
df_t = transform.lag_dv(df_t, 'emails', 3, 'systemid')
df_t = transform.lag_dv(df_t, 'phones', 3, 'systemid')
df_t = transform.lag_dv(df_t, 'ftf', 3, 'systemid')
df_t = transform.lag_dv(df_t, 'virtual', 3, 'systemid')
df_t = transform.lag_dv(df_t, 'mssg', 3, 'systemid')
df_t = transform.lag_dv(df_t, 'other', 3, 'systemid')
df_t

,systemid,date,basezip,systemname,emails,phones,ftf,virtual,mssg,other,...,ftf_lag3,virtual_lag1,virtual_lag2,virtual_lag3,mssg_lag1,mssg_lag2,mssg_lag3,other_lag1,other_lag2,other_lag3
0,4769,2022-10,29732.0,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4769,2022-11,29732.0,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4769,2022-12,29732.0,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4769,2023-01,29732.0,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",2.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4769,2023-02,29732.0,"CAROLINA BLOOD AND CANCER CARE ASSOCIATES, PA",1.0,2.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8775,44280,2023-03,78229.0,"SOUTH TEXAS ONCOLOGY & HEMATOLOGY, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8776,44280,2023-04,78229.0,"SOUTH TEXAS ONCOLOGY & HEMATOLOGY, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8777,44280,2023-05,78229.0,"SOUTH TEXAS ONCOLOGY & HEMATOLOGY, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8778,44280,2023-06,78229.0,"SOUTH TEXAS ONCOLOGY & HEMATOLOGY, PA",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
df_t.corr()

,systemid,basezip,emails,phones,ftf,virtual,mssg,other,quantity_adj,dds_in_pd_cnt,...,ftf_lag3,virtual_lag1,virtual_lag2,virtual_lag3,mssg_lag1,mssg_lag2,mssg_lag3,other_lag1,other_lag2,other_lag3
systemid,1.000000,-0.043763,0.037912,-0.004015,-0.008886,0.019179,0.029247,0.000058,0.000115,-0.009650,...,-0.004148,0.014108,0.012320,0.014456,0.032976,0.034548,0.032095,0.002737,0.015993,0.019747
basezip,-0.043763,1.000000,0.076187,-0.035517,0.013940,-0.030617,-0.013541,-0.056003,-0.012282,-0.019945,...,0.030942,-0.026328,-0.023861,-0.027322,-0.008357,-0.005584,-0.001913,-0.056191,-0.066771,-0.067540
emails,0.037912,0.076187,1.000000,0.387262,0.358599,0.282910,0.074155,0.034682,0.171971,0.062356,...,0.235979,0.155933,0.143641,0.098248,0.068746,0.048191,0.096907,0.024154,0.021577,0.074092
phones,-0.004015,-0.035517,0.387262,1.000000,0.317963,0.171426,0.062886,0.003224,0.102287,0.058069,...,0.155250,0.114231,0.056955,0.067330,0.030826,0.017036,0.016439,0.011754,0.015131,0.023222
ftf,-0.008886,0.013940,0.358599,0.317963,1.000000,0.114316,0.067133,-0.009802,0.201242,0.043905,...,0.216584,0.069925,0.074945,0.031985,0.049675,0.026840,0.068419,0.002437,0.019967,0.012953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mssg_lag2,0.034548,-0.005584,0.048191,0.017036,0.026840,0.026657,0.388849,-0.005704,-0.000451,0.007872,...,0.041874,0.063926,0.079888,0.009939,0.391694,1.000000,0.347443,-0.007747,0.006960,-0.006903
mssg_lag3,0.032095,-0.001913,0.096907,0.016439,0.068419,0.038733,0.307427,0.004502,0.020571,0.002279,...,0.052894,0.036099,0.073115,0.091948,0.370274,0.347443,1.000000,-0.013755,-0.011882,0.014300
other_lag1,0.002737,-0.056191,0.024154,0.011754,0.002437,0.029660,-0.015007,0.193214,-0.008394,0.134683,...,0.016642,0.036804,0.024515,0.015405,0.005480,-0.007747,-0.013755,1.000000,0.221857,0.124460
other_lag2,0.015993,-0.066771,0.021577,0.015131,0.019967,-0.010890,-0.009042,0.105354,-0.014860,0.104748,...,0.016763,0.039157,0.045256,0.036159,-0.010812,0.006960,-0.011882,0.221857,1.000000,0.227803


# Final Model Fitting

In [21]:
modeling = mmm_modeling.MMMModeling()

In [134]:
channels = ['emails', 'phones', 'ftf', 'virtual', 'mssg', 'other']
lag_dv = [x for x in df_t.columns if 'quantity_adj_lag' in x]
lag_channels = [x for x in df_t.columns if ('lag' in x) & ('quantity_adj' not in x)]
non_media = df_t.drop(channels + lag_dv + lag_channels + ['quantity_adj', 'systemid', 'date', 'basezip', 'systemname'], axis=1).columns.tolist()
#X = df_t[channels + lag_channels + lag_dv]
X = df_t[channels + lag_channels + lag_dv + non_media]
y = df_t['quantity_adj']
model = modeling.rf_regressor(df_t, X.columns.tolist(), 'quantity_adj', 'date')

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_modeling.py:62: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_modeling.py:63: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [135]:
# performance
model['performance']

{'full': {'r2': 0.8149142924326336,
  'rmse': 4.902280325252752,
  'mape': 0.7611652541431498},
 'train': {'r2': 0.7945625436653243,
  'rmse': 5.005837636659457,
  'mape': 0.9562686897424938},
 'test': {'r2': -0.03079198784116821,
  'rmse': 12.865220762215548,
  'mape': 2.47601336027041}}

In [136]:
# importance
model['importance'].head(50)

,feature,importance,std
26,quantity_adj_lag3,0.483867,0.160617
24,quantity_adj_lag1,0.169639,0.117612
25,quantity_adj_lag2,0.093078,0.125739
2,ftf,0.032168,0.036563
7,emails_lag2,0.026398,0.037081
0,emails,0.024597,0.030972
12,ftf_lag1,0.024151,0.030866
1,phones,0.020396,0.030160
6,emails_lag1,0.020264,0.022501
11,phones_lag3,0.015334,0.024211


In [65]:
model['importance'].to_csv('imp.csv', index=False)

# Response Curves

In [28]:
response_curves = mmm_response_curves.MMMResponseCurves()

In [140]:
channels = ['emails', 'phones', 'ftf', 'virtual', 'mssg', 'other']
lag_dv = [x for x in df_t.columns if 'quantity_adj_lag' in x]
lag_channels = [x for x in df_t.columns if ('lag' in x) & ('quantity_adj' not in x)]
non_media = df_t.drop(channels + lag_dv + lag_channels + ['quantity_adj', 'systemid', 'date', 'basezip', 'systemname'], axis=1).columns.tolist()
X = df_t[channels + lag_channels + lag_dv + non_media]

In [94]:
X.describe()

,emails,phones,ftf,virtual,mssg,other,emails_lag1,emails_lag2,emails_lag3,phones_lag1,...,avgoccrate,routineday,h_pdays,avglos,mparmix,mcaredrg,ttldrg,mcr_alos,hdrg,mcr_pdays
count,8780.000000,8780.000000,8780.000000,8780.000000,8780.000000,8780.000000,8780.000000,8780.000000,8780.000000,8780.000000,...,8780.000000,8.780000e+03,8.780000e+03,8780.000000,8780.000000,8780.000000,8.780000e+03,8780.000000,8.780000e+03,8.780000e+03
mean,0.343964,0.142027,0.117882,0.025626,0.013440,0.070843,0.297494,0.222323,0.187016,0.115945,...,2.658833,2.323877e+05,2.323877e+05,37.749661,2.759348,14664.210626,5.719479e+04,39.788235,5.719479e+04,6.891180e+04
std,1.332655,0.685213,0.561209,0.188287,0.179358,0.328253,1.265386,0.917860,0.831992,0.616067,...,3.235790,2.788143e+05,2.788143e+05,63.987459,3.430046,17629.717248,7.158559e+04,56.341959,7.158559e+04,8.001317e+04
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.036300,5.530000e+02,5.530000e+02,1.522800,0.003700,5.000000,7.600000e+01,1.000000,7.600000e+01,1.700000e+01
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.487900,1.759000e+05,1.759000e+05,18.521900,1.422500,9998.000000,4.228800e+04,22.000000,4.228800e+04,4.807800e+04
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.660000,2.323877e+05,2.323877e+05,37.750000,2.760000,14664.210000,5.719479e+04,39.790000,5.719479e+04,6.891180e+04
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.660000,2.323877e+05,2.323877e+05,37.750000,2.760000,14664.210000,5.719479e+04,39.790000,5.719479e+04,6.891180e+04
max,58.000000,20.000000,14.000000,4.000000,6.000000,6.000000,58.000000,22.000000,18.000000,20.000000,...,50.049800,5.414923e+06,5.414923e+06,1238.712500,49.508900,334041.000000,1.424002e+06,1069.000000,1.424002e+06,1.466733e+06


In [145]:
channel1 = response_curves.responses(model['full_model'], X, 'emails', 30, 1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:04<00:00,  6.90it/s]
D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power



In [146]:
response_curves.plot(channel1['resp_df'], 'touches', ['emails', 'emails_hill_estimate'])

In [59]:
channel1['resp_df'].to_csv('spectrum_emails.csv', index=False)

In [126]:
channel2 = response_curves.responses(model['full_model'], X, 'phones', 18, 1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:02<00:00,  7.47it/s]
D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power



In [128]:
response_curves.plot(channel2['resp_df'], 'touches', ['phones', 'phones_hill_estimate'])

In [60]:
channel2['resp_df'].to_csv('spectrum_phones.csv', index=False)

In [147]:
channel3 = response_curves.responses(model['full_model'], X, 'ftf', 12, 1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:01<00:00,  7.17it/s]
D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in reciprocal

D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:24: RuntimeWarning:

divide by zero encountered in power



In [148]:
response_curves.plot(channel3['resp_df'], 'touches', ['ftf', 'ftf_hill_estimate'])

In [61]:
channel3['resp_df'].to_csv('spectrum_ftf.csv', index=False)

In [53]:
channel4 = response_curves.responses(model['full_model'], X, 'virtual', 20, 0.1)

 47%|████████████████████████████████████████████████████████████▊                                                                    | 99/210 [00:15<00:16,  6.88it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 48%|████████████████████████████████████████████████████████████▉                                                                   | 100/210 [00:15<00:15,  7.05it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 54%|█████████████████████████████████████████████████████████████████████▍                                                          | 114/210 [00:18<00:16,  5.77it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 55%|██████████████████████████████████████████████████████████████████████                                                          | 115/210 [00:18<00:15,  6.07it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 61%|██████████████████████████████████████████████████████████████████████████████▋                                                 | 129/210 [00:20<00:12,  6.47it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 62%|███████████████████████████████████████████████████████████████████████████████▏                                                | 130/210 [00:20<00:12,  6.27it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 69%|███████████████████████████████████████████████████████████████████████████████████████▊                                        | 144/210 [00:23<00:10,  6.47it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 69%|████████████████████████████████████████████████████████████████████████████████████████▍                                       | 145/210 [00:23<00:10,  6.28it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 76%|████████████████████████████████████████████████████████████████████████████████████████████████▉                               | 159/210 [00:25<00:08,  6.13it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 76%|█████████████████████████████████████████████████████████████████████████████████████████████████▌                              | 160/210 [00:25<00:07,  6.50it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 83%|██████████████████████████████████████████████████████████████████████████████████████████████████████████                      | 174/210 [00:27<00:05,  6.87it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 83%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▋                     | 175/210 [00:27<00:05,  6.81it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏            | 189/210 [00:29<00:02,  7.24it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊            | 190/210 [00:29<00:02,  7.20it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎   | 204/210 [00:32<00:00,  6.51it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉   | 205/210 [00:32<00:00,  6.35it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

In [54]:
response_curves.plot(channel4['resp_df'], 'touches', ['virtual', 'virtual_hill_estimate'])

In [62]:
channel4['resp_df'].to_csv('spectrum_virtual.csv', index=False)

In [55]:
channel5 = response_curves.responses(model['full_model'], X, 'mssg', 20, 0.1)

 47%|████████████████████████████████████████████████████████████▊                                                                    | 99/210 [00:20<00:23,  4.73it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 48%|████████████████████████████████████████████████████████████▉                                                                   | 100/210 [00:20<00:23,  4.61it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 54%|█████████████████████████████████████████████████████████████████████▍                                                          | 114/210 [00:23<00:19,  4.81it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 55%|██████████████████████████████████████████████████████████████████████                                                          | 115/210 [00:24<00:23,  4.09it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 61%|██████████████████████████████████████████████████████████████████████████████▋                                                 | 129/210 [00:27<00:15,  5.11it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 62%|███████████████████████████████████████████████████████████████████████████████▏                                                | 130/210 [00:27<00:15,  5.06it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 69%|███████████████████████████████████████████████████████████████████████████████████████▊                                        | 144/210 [00:30<00:13,  4.78it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 69%|████████████████████████████████████████████████████████████████████████████████████████▍                                       | 145/210 [00:30<00:13,  4.85it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 76%|████████████████████████████████████████████████████████████████████████████████████████████████▉                               | 159/210 [00:33<00:11,  4.32it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 76%|█████████████████████████████████████████████████████████████████████████████████████████████████▌                              | 160/210 [00:34<00:11,  4.31it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 83%|██████████████████████████████████████████████████████████████████████████████████████████████████████████                      | 174/210 [00:37<00:07,  5.13it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 83%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▋                     | 175/210 [00:37<00:06,  5.17it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏            | 189/210 [00:40<00:04,  4.97it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊            | 190/210 [00:40<00:03,  5.11it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎   | 204/210 [00:42<00:01,  5.39it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉   | 205/210 [00:43<00:00,  5.02it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

In [56]:
response_curves.plot(channel5['resp_df'], 'touches', ['mssg', 'mssg_hill_estimate'])

In [63]:
channel5['resp_df'].to_csv('spectrum_mssg.csv', index=False)

In [57]:
channel6 = response_curves.responses(model['full_model'], X, 'other', 20, 0.1)

 47%|████████████████████████████████████████████████████████████▊                                                                    | 99/210 [00:16<00:21,  5.15it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 48%|████████████████████████████████████████████████████████████▉                                                                   | 100/210 [00:16<00:20,  5.39it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 54%|█████████████████████████████████████████████████████████████████████▍                                                          | 114/210 [00:18<00:14,  6.56it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 55%|██████████████████████████████████████████████████████████████████████                                                          | 115/210 [00:19<00:14,  6.67it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 61%|██████████████████████████████████████████████████████████████████████████████▋                                                 | 129/210 [00:21<00:12,  6.36it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 62%|███████████████████████████████████████████████████████████████████████████████▏                                                | 130/210 [00:21<00:12,  6.51it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 69%|███████████████████████████████████████████████████████████████████████████████████████▊                                        | 144/210 [00:23<00:10,  6.54it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 69%|████████████████████████████████████████████████████████████████████████████████████████▍                                       | 145/210 [00:23<00:10,  6.43it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 76%|████████████████████████████████████████████████████████████████████████████████████████████████▉                               | 159/210 [00:26<00:09,  5.44it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 76%|█████████████████████████████████████████████████████████████████████████████████████████████████▌                              | 160/210 [00:26<00:08,  5.87it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 83%|██████████████████████████████████████████████████████████████████████████████████████████████████████████                      | 174/210 [00:28<00:05,  6.29it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 83%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▋                     | 175/210 [00:28<00:05,  6.45it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏            | 189/210 [00:31<00:03,  5.54it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊            | 190/210 [00:31<00:03,  5.53it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

 97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎   | 204/210 [00:33<00:00,  6.48it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`

 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉   | 205/210 [00:33<00:00,  6.64it/s]D:\Users\hartsingh\Documents\Projects\promotional_response_rf\research\mmm_response_curves.py:139: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

In [58]:
response_curves.plot(channel6['resp_df'], 'touches', ['other', 'other_hill_estimate'])

In [64]:
channel6['resp_df'].to_csv('spectrum_other.csv', index=False)

# Channel 1 Optimization

In [57]:
p1_hill = pd.DataFrame(channel1_segment['optimal_hill']).T.reset_index()
p1_hill.columns = ['feature', 'beta', 'gamma', 'alpha']
p1_hill

,feature,beta,gamma,alpha
0,Specialty_AC_P1_Arikayce,205.637938,51.313808,4.712360
1,Specialty_ADU_P1_Arikayce,205.517174,51.329996,4.719966
2,Specialty_CCE_P1_Arikayce,204.495296,51.466856,4.785168
3,Specialty_CCM_P1_Arikayce,184.168059,53.853232,6.000614
4,Specialty_EM_P1_Arikayce,205.726333,51.301955,4.706769
5,Specialty_FM_P1_Arikayce,204.875774,51.678124,5.028733
6,Specialty_GP_P1_Arikayce,203.352425,51.619572,4.858920
7,Specialty_HOS_P1_Arikayce,205.886561,51.280453,4.696636
8,Specialty_ID_P1_Arikayce,234.191120,48.921949,5.973733
9,Specialty_IM_P1_Arikayce,207.176496,49.692169,3.784883


In [58]:
# hyperopt hill
optimization = mmm_optimization.MMMOptimization(budget=354, params=p1_hill)
channels = p1_hill['feature'].tolist()
output = optimization.optimize_hyperopt_hill(channels, 2000)
output

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:10<00:00, 28.31trial/s, best loss: 99997133.878953]


{'mix': {'Specialty_AC_P1_Arikayce': 330,
  'Specialty_ADU_P1_Arikayce': 223,
  'Specialty_CCE_P1_Arikayce': 194,
  'Specialty_CCM_P1_Arikayce': 340,
  'Specialty_EM_P1_Arikayce': 174,
  'Specialty_FM_P1_Arikayce': 171,
  'Specialty_GP_P1_Arikayce': 250,
  'Specialty_HOS_P1_Arikayce': 222,
  'Specialty_ID_P1_Arikayce': 266,
  'Specialty_IM_P1_Arikayce': 211,
  'Specialty_Other_P1_Arikayce': 330,
  'Specialty_PCC_P1_Arikayce': 188,
  'Specialty_PDP_P1_Arikayce': 252,
  'Specialty_PUD_P1_Arikayce': 221},
 'trials': [{'loss': 99997911.78291155, 'status': 'ok'},
  {'loss': 99997367.74888934, 'status': 'ok'},
  {'loss': 99997252.43217513, 'status': 'ok'},
  {'loss': 99997818.15737288, 'status': 'ok'},
  {'loss': 99997880.892491, 'status': 'ok'},
  {'loss': 99997516.23347887, 'status': 'ok'},
  {'loss': 99997876.8572574, 'status': 'ok'},
  {'loss': 99997331.26427692, 'status': 'ok'},
  {'loss': 99997140.25295413, 'status': 'ok'},
  {'loss': 99997584.77514112, 'status': 'ok'},
  {'loss': 9999

# ID Budgeting

In [71]:
optimization = mmm_optimization.MMMOptimization(budget=135)
channels = ['P1_Arikayce', 'P2_Arikayce']
specialties = [x for x in df_t.columns if x.startswith('Specialty_')]
lag_dv = [x for x in df_t.columns if 'rx_count_lag' in x]
lag_channels = [x for x in df_t.columns if ('lag' in x) & ('rx_count' not in x)]
X = df_t[channels + lag_channels + specialties]
X[specialties] = 0
X['Specialty_ID'] = 1
output = optimization.optimize_predict(X, channels, 2000, model['full_model'])
output

  0%|                                                                                                                             | 0/2000 [00:00<?, ?trial/s, best loss=?]

D:\Users\hartsingh\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:3645: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\program files (x86)\python\python37-32\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:02<00:00, 31.98trial/s, best loss: -264.04118068466823]


{'mix': {'P1_Arikayce': 90, 'P2_Arikayce': 3},
 'trials': [{'loss': 99999740.31881931, 'status': 'ok'},
  {'loss': -123.15118068466835, 'status': 'ok'},
  {'loss': 99999738.89881931, 'status': 'ok'},
  {'loss': 99999785.74881932, 'status': 'ok'},
  {'loss': -34.57118068466833, 'status': 'ok'},
  {'loss': -126.65118068466835, 'status': 'ok'},
  {'loss': -6.371180684668324, 'status': 'ok'},
  {'loss': -260.7011806846684, 'status': 'ok'},
  {'loss': 99999740.31881931, 'status': 'ok'},
  {'loss': 99999740.65881932, 'status': 'ok'},
  {'loss': -123.15118068466835, 'status': 'ok'},
  {'loss': -11.851180684668325, 'status': 'ok'},
  {'loss': 99999732.60881932, 'status': 'ok'},
  {'loss': 99999741.44881931, 'status': 'ok'},
  {'loss': 99999732.56881931, 'status': 'ok'},
  {'loss': -19.181180684668327, 'status': 'ok'},
  {'loss': -124.69118068466831, 'status': 'ok'},
  {'loss': 99999872.34881932, 'status': 'ok'},
  {'loss': 99999740.15881932, 'status': 'ok'},
  {'loss': 99999952.48881932, 'stat

# Overall Budget

In [74]:
optimization = mmm_optimization.MMMOptimization(budget=368)
channels = ['P1_Arikayce', 'P2_Arikayce']
specialties = [x for x in df_t.columns if x.startswith('Specialty_')]
lag_dv = [x for x in df_t.columns if 'rx_count_lag' in x]
lag_channels = [x for x in df_t.columns if ('lag' in x) & ('rx_count' not in x)]
X = df_t[channels + lag_channels + specialties]
output = optimization.optimize_predict(X, channels, 2000, model['full_model'])
output

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:01<00:00, 32.58trial/s, best loss: -271.404799150921]


{'mix': {'P1_Arikayce': 288, 'P2_Arikayce': 13},
 'trials': [{'loss': 99999736.8873437, 'status': 'ok'},
  {'loss': 99999778.46091513, 'status': 'ok'},
  {'loss': -268.7090848652068, 'status': 'ok'},
  {'loss': -255.94622772234968, 'status': 'ok'},
  {'loss': -221.10908486520682, 'status': 'ok'},
  {'loss': 99999758.573058, 'status': 'ok'},
  {'loss': 99999784.95734371, 'status': 'ok'},
  {'loss': 99999743.05377227, 'status': 'ok'},
  {'loss': 99999743.05377227, 'status': 'ok'},
  {'loss': -8.623370579492526, 'status': 'ok'},
  {'loss': -195.39979915092113, 'status': 'ok'},
  {'loss': 99999790.75448656, 'status': 'ok'},
  {'loss': -262.32837057949257, 'status': 'ok'},
  {'loss': -33.75122772234967, 'status': 'ok'},
  {'loss': 99999736.60591513, 'status': 'ok'},
  {'loss': -208.24551343663535, 'status': 'ok'},
  {'loss': 99999743.05377227, 'status': 'ok'},
  {'loss': 99999743.05377227, 'status': 'ok'},
  {'loss': -218.0883705794926, 'status': 'ok'},
  {'loss': -207.00408486520678, 'stat

In [ ]:
# dont need lag dv - point is to estimate the impact of channels as best as possible and that happens when their importance is high